In [9]:
# Install required packages.
!pip install node2vec
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
Looking in links: https://data.pyg.org/whl/nightly/torch-2.5.1+cu124.html
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-c2r02nb9
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-c2r02nb9
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit bb6601c8666e69205a7a4d1d981b771e9bae6880
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import random
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from node2vec import Node2Vec

def mask_edges(G, mask_fraction=0.2, seed=42):
    """
    Randomly remove a fraction of edges from G for training,
    and return the masked graph, the list of removed edges (positives),
    and an equal number of negative edge samples (non-edges).
    """
    np.random.seed(seed)
    G_train = G.copy()
    all_edges = list(G.edges())
    num_mask = int(mask_fraction * len(all_edges))
    masked_edges = random.sample(all_edges, num_mask)

    # Remove masked edges from the training graph
    G_train.remove_edges_from(masked_edges)

    # Generate negative samples: sample node pairs that are not in G
    non_edges = list(nx.non_edges(G))
    negative_edges = random.sample(non_edges, num_mask)

    return G_train, masked_edges, negative_edges

G = nx.read_graphml("country_graph.graphml")  # Directed graph
G = G.to_directed()

# Mask 20% of the edges
G_train, positive_edges, negative_edges = mask_edges(G, mask_fraction=0.2)

# Generate Node2vec embeddings
node2vec = Node2Vec(G_train, dimensions=64, walk_length=30, num_walks=200, workers=2, seed=42)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Cosine similarity function
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# Evaluate link prediction using Node2vec embeddings
def evaluate_link_prediction(model, pos_edges, neg_edges):
    # y_true: 1 for positive edges, 0 for negative edges
    y_true = [1] * len(pos_edges) + [0] * len(neg_edges)
    scores = []
    for u, v in pos_edges + neg_edges:
        # Use string conversion to match the model vocabulary keys.
        if str(u) in model.wv and str(v) in model.wv:
            score = cosine_similarity(model.wv[str(u)], model.wv[str(v)])
        else:
            score = 0
        scores.append(score)
    auc = roc_auc_score(y_true, scores)
    return auc

auc_node2vec = evaluate_link_prediction(model, positive_edges, negative_edges)
print("Node2vec Link Prediction ROC AUC:", auc_node2vec)


Computing transition probabilities:   0%|          | 0/249 [00:00<?, ?it/s]

Node2vec Link Prediction ROC AUC: 0.9229579732801615


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import from_networkx, negative_sampling, train_test_split_edges
from torch_geometric.nn import GCNConv
import string
from sklearn.metrics import roc_auc_score

# Convert the NetworkX graph to a PyG Data object.
G_undirected = G.to_undirected()
data = from_networkx(G_undirected)

# Create one-hot encoding of first and last letters.
def create_node_features(G):
    features = []
    letters = list(string.ascii_uppercase)
    for node in G.nodes():
        name = str(node).upper()
        feat = [0] * (2 * len(letters))
        if len(name) > 0:
            if name[0] in letters:
                feat[letters.index(name[0])] = 1  # first letter one-hot
            if name[-1] in letters:
                feat[len(letters) + letters.index(name[-1])] = 1  # last letter one-hot
        features.append(feat)
    return torch.tensor(features, dtype=torch.float)
data.x = create_node_features(G_undirected)

# split edges into train/test sets.
data = train_test_split_edges(data)

class GCNLinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNLinkPredictor, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode(self, z, edge_index):
        src, dst = edge_index
        return (z[src] * z[dst]).sum(dim=1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

model = GCNLinkPredictor(in_channels=data.x.shape[1], hidden_channels=32, out_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Unsupervised link prediction loss: margin ranking loss.
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.train_pos_edge_index)

    pos_score = model.decode(z, data.train_pos_edge_index)
    neg_edge_index = negative_sampling(
        edge_index=data.train_pos_edge_index, num_nodes=data.num_nodes,
        num_neg_samples=data.train_pos_edge_index.size(1))
    neg_score = model.decode(z, neg_edge_index)

    target = torch.ones(pos_score.size())
    loss = F.margin_ranking_loss(pos_score, neg_score, target, margin=1.0)
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test():
    model.eval()
    z = model.encode(data.x, data.train_pos_edge_index)
    pos_score = model.decode(z, data.test_pos_edge_index)
    neg_score = model.decode(z, data.test_neg_edge_index)
    y_true = torch.cat([torch.ones(pos_score.size(0)), torch.zeros(neg_score.size(0))])
    scores = torch.cat([pos_score, neg_score])
    auc = roc_auc_score(y_true.numpy(), scores.numpy())
    return auc

print("Seed: 42, Negative edges in training set:", data.train_pos_edge_index.size(1))

# Train the model for 200 epochs; every 20 epochs, print epoch, loss, and Test AUC.
print("Training GNN-based Link Prediction Model:")
for epoch in range(1, 201):
    loss = train()
    if epoch % 20 == 0:
        auc = test()
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, Test AUC: {auc:.4f}")


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Seed: 42, Negative edges in training set: 5266
Training GNN-based Link Prediction Model:
Epoch: 020, Loss: 0.2824, Test AUC: 0.8747
Epoch: 040, Loss: 0.1515, Test AUC: 0.9163
Epoch: 060, Loss: 0.1126, Test AUC: 0.9392
Epoch: 080, Loss: 0.0876, Test AUC: 0.9527
Epoch: 100, Loss: 0.0803, Test AUC: 0.9567
Epoch: 120, Loss: 0.0773, Test AUC: 0.9582
Epoch: 140, Loss: 0.0723, Test AUC: 0.9588
Epoch: 160, Loss: 0.0609, Test AUC: 0.9578
Epoch: 180, Loss: 0.0656, Test AUC: 0.9594
Epoch: 200, Loss: 0.0722, Test AUC: 0.9574
